In [ ]:
μ_min1 = Array{Float64}(undef, 2, 2, 2, 2)
π0 = Array{Float64}(undef, 2 ,2 , 2, 2)
β = 0.8
ν = 1.1

In [ ]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        μ_min1[i, j, :, :] = matrix2
    end
end
μ_min1

In [ ]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        π0[i, j, :, :] = matrix2
    end
end
π0

In [ ]:
κdot = rand(0.95:0.001:1.08, (2, 2, 2, 2))

In [ ]:
Ȧ = rand(0.98:0.001:1.05, (2, 2))

In [ ]:
L_setup_matrix = rand(2, 2)
L0 = L_setup_matrix/sum(L_setup_matrix)

In [ ]:
w0 = rand(0.5:0.001:1.5, (2, 2))

In [ ]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, 2)

In [ ]:
u̇_0t = Array{Float64}(undef, 2, 2, 2)
u̇_0t[:, :, 1] = rand(0.98:0.001:1.02, (2, 2))
u̇_0t[:, :, 2] = ones(2, 2)

In [ ]:
ẇ = rand(0.97:0.001:1.06, (2, 2))

In [ ]:
μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
L1 = Array{Float64}(undef, 2, 2)
ẇn = Array{Float64}(undef, 2, 2)
Ṗ = Array{Float64}(undef, 2, 2)
π1 = Array{Float64}(undef, 2, 2, 2, 2)
X = Array{Float64}(undef, 2, 2)
α = Array{Float64}(undef, 2)
Xt1 = Array{Float64}(undef, 2, 2)

In [ ]:
while !all(0.99 .* u̇_0t[:, :, 2] .<= u̇_0t[:, :, 1] .<= 1.01 .* u̇_0t[:, :, 2])
#    μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
    for n in 1:2
        for j in 1:2
            for i in 1:2
                for k in 1:2
                    denom = sum(μ_min1[n, j, m, h]*((u̇_0t[m, h, 2])^(β/ν)) for m in 1:2, h in 1:2)
                    μ_0[n, j, i, k] = (μ_min1[n, j, i, k]*((u̇_0t[i, k, 2])^(β/ν)))/denom
                    println(μ_0[n, j, i, k], "_mu_0")
                end
            end
        end
    end
#    L1 = Array{Float64}(undef, 2, 2)
    for n in 1:2
        for j in 1:2
            L1[n, j] = sum(μ_0[i, k, n, j]*L0[i, k] for i in 1:2, k in 1:2)
            println("_L1")
        end
    end
#    ẇn = Array{Float64}(undef, 2, 2)
    while !all(0.99*ẇ .<= ẇn .<= 1.01*ẇ)

#        Ṗ = Array{Float64}(undef, 2, 2)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    Ṗ[n, j] = (sum(π0[n, j, i, j]*((ẇ[i, j]*κdot[n, j, i, j])^(-θ_list[j]))*((Ȧ[i, j])^θ_list[j]) for i in 1:2))^(-1/θ_list[j])
                    println("_Pdot")    
                end
            end
        end
#        π1 = Array{Float64}(undef, 2, 2, 2, 2)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    π1[n, j, i, j] = π0[n, j, i, j]*(((ẇ[i, j]*κdot[n, j, i, j])/Ṗ[n, j])^(-θ_list[j]))*((Ȧ[i,j])^θ_list[j])
                    println("_pi1")
                end
            end
        end
        for i in 1:2
            for j in 1:2
                X[i, j] = w0[i, j]*L0[i, j]
                println("_w0")
            end
        end
        for i in 1:2
            α[i] = sum(X[k, i] for k in 1:2)/sum(X[j, l] for j in 1:2, l in 1:2)
            println("_alpha")
        end
#        Xt1 = Array{Float64}(undef, 2, 2)
        for n in 1:2
            for j in 1:2
                Xt1[n, j] = α[j]*sum(ẇ[n, k]*L1[n, k]*w0[n, k] for k in 1:2)
                println("_Xt1")
            end
        end
        for n in 1:2
            for j in 1:2
                ẇn[n, j] = sum(π1[i, j, n, j]*Xt1[i, j] for i in 1:2)/(L1[n, j]*w0[n, j])
                println("_wdotn")
            end
        end
        ẇ = ẇn
        println("_w reassigned")
    end
    u̇_1t = Array{Float64}(undef, 2, 2)
    for n in 1:2
        for j in 1:2
            u̇_1t[n, j] = ẇ[n, j]*(sum(μ_0[n, j, i, k]*(u̇_0t[i, k, 2])^(β/ν) for i in 1:2, k in 1:2))^(ν)
            println(u̇_1t[n, j])
        end
    end
    println(u̇_0t)
    u̇_0t[:, :, 1] = u̇_1t
    println("_udot0_updated")
    println(u̇_1t)
end 